In [30]:
import requests
import credentials
import pandas as pd


In [31]:
session = requests.Session()

In [32]:
# we read the available pairs from the file we have saved before

instrumnet_df = pd.read_pickle('instruments.pkl')

In [33]:
# the currencies we are interested to trade them

our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [34]:
instrumnet_df.shape

(68, 5)

In [35]:
instrumnet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [36]:
instrumnet_df.columns

Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [37]:
instrumnet_df.name.unique()

array(['EUR_DKK', 'USD_MXN', 'GBP_USD', 'CAD_CHF', 'EUR_GBP', 'GBP_CHF',
       'USD_THB', 'USD_ZAR', 'EUR_NOK', 'GBP_PLN', 'GBP_CAD', 'CAD_HKD',
       'EUR_HKD', 'CHF_JPY', 'USD_SEK', 'GBP_SGD', 'GBP_HKD', 'EUR_NZD',
       'SGD_CHF', 'AUD_SGD', 'EUR_JPY', 'USD_CHF', 'USD_TRY', 'GBP_JPY',
       'EUR_CZK', 'EUR_TRY', 'USD_JPY', 'GBP_ZAR', 'SGD_JPY', 'USD_CZK',
       'ZAR_JPY', 'TRY_JPY', 'USD_DKK', 'EUR_PLN', 'AUD_CAD', 'USD_HKD',
       'CAD_SGD', 'GBP_NZD', 'CHF_ZAR', 'USD_CNH', 'NZD_USD', 'EUR_HUF',
       'AUD_NZD', 'NZD_HKD', 'USD_NOK', 'NZD_SGD', 'CAD_JPY', 'HKD_JPY',
       'AUD_CHF', 'NZD_CHF', 'EUR_SEK', 'USD_SGD', 'AUD_HKD', 'AUD_JPY',
       'EUR_ZAR', 'EUR_AUD', 'NZD_JPY', 'CHF_HKD', 'USD_CAD', 'EUR_CHF',
       'EUR_CAD', 'USD_HUF', 'NZD_CAD', 'EUR_SGD', 'AUD_USD', 'EUR_USD',
       'GBP_AUD', 'USD_PLN'], dtype=object)

In [38]:
# fetch candles from the api
def fetch_candles(pair_name, count, granularity):
    url = f"{credentials.OANDA_URL}/instruments/{pair_name}/candles"
    paramas = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=paramas, headers=credentials.SECURE_HEADER)
    return response.status_code, response.json()

In [39]:
# this is to test the function
code, res= fetch_candles("EUR_USD", 10, "H1")
print(code)
print(res)

200
{'instrument': 'EUR_USD', 'granularity': 'H1', 'candles': [{'complete': True, 'volume': 3193, 'time': '2022-06-20T15:00:00.000000000Z', 'bid': {'o': '1.05342', 'h': '1.05376', 'l': '1.05230', 'c': '1.05373'}, 'mid': {'o': '1.05350', 'h': '1.05383', 'l': '1.05237', 'c': '1.05380'}, 'ask': {'o': '1.05358', 'h': '1.05390', 'l': '1.05244', 'c': '1.05388'}}, {'complete': True, 'volume': 2244, 'time': '2022-06-20T16:00:00.000000000Z', 'bid': {'o': '1.05373', 'h': '1.05375', 'l': '1.05186', 'c': '1.05227'}, 'mid': {'o': '1.05381', 'h': '1.05383', 'l': '1.05193', 'c': '1.05234'}, 'ask': {'o': '1.05389', 'h': '1.05391', 'l': '1.05200', 'c': '1.05242'}}, {'complete': True, 'volume': 963, 'time': '2022-06-20T17:00:00.000000000Z', 'bid': {'o': '1.05226', 'h': '1.05226', 'l': '1.05050', 'c': '1.05051'}, 'mid': {'o': '1.05233', 'h': '1.05233', 'l': '1.05060', 'c': '1.05060'}, 'ask': {'o': '1.05240', 'h': '1.05241', 'l': '1.05068', 'c': '1.05069'}}, {'complete': True, 'volume': 1459, 'time': '202

In [40]:
# creates a data frame from a response from api
def get_candles_df(json_response):
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [41]:
# these two cells are for testing the function
code, res = fetch_candles("EUR_USD", 10, "H1")

In [42]:
df = get_candles_df(res)
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-06-20T15:00:00.000000000Z,3193,1.05350,1.05383,1.05237,1.05380,1.05342,1.05376,1.05230,1.05373,1.05358,1.05390,1.05244,1.05388
1,2022-06-20T16:00:00.000000000Z,2244,1.05381,1.05383,1.05193,1.05234,1.05373,1.05375,1.05186,1.05227,1.05389,1.05391,1.05200,1.05242
2,2022-06-20T17:00:00.000000000Z,963,1.05233,1.05233,1.05060,1.05060,1.05226,1.05226,1.05050,1.05051,1.05240,1.05241,1.05068,1.05069
3,2022-06-20T18:00:00.000000000Z,1459,1.05062,1.05074,1.04990,1.04992,1.05053,1.05067,1.04981,1.04983,1.05070,1.05082,1.04997,1.05000
4,2022-06-20T19:00:00.000000000Z,809,1.04996,1.05122,1.04981,1.05114,1.04987,1.05112,1.04972,1.05105,1.05004,1.05131,1.04989,1.05123
5,2022-06-20T20:00:00.000000000Z,796,1.05119,1.05146,1.05090,1.05112,1.05109,1.05136,1.05080,1.05099,1.05129,1.05156,1.05099,1.05124
6,2022-06-20T21:00:00.000000000Z,286,1.05088,1.05150,1.05086,1.05116,1.05049,1.05100,1.05044,1.05078,1.05128,1.05200,1.05120,1.05155
7,2022-06-20T22:00:00.000000000Z,1095,1.05117,1.05188,1.05090,1.05184,1.05075,1.05180,1.05075,1.05175,1.05159,1.05196,1.05099,1.05194
8,2022-06-20T23:00:00.000000000Z,1879,1.05184,1.05212,1.05138,1.05175,1.05176,1.05204,1.05130,1.05167,1.05193,1.05220,1.05146,1.05183


In [43]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"hist_data/{pair}_{granularity}.pkl")

In [44]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} laoded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [45]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in instrumnet_df.name.unique():
            create_data(pair, "H1")

EUR_USD laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
EUR_GBP laoded 3999 candles from 2021-10-29T02:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
EUR_JPY laoded 3999 candles from 2021-10-28T20:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
EUR_CHF laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
EUR_NZD laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
EUR_CAD laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
USD_JPY laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
USD_CHF laoded 3999 candles from 2021-10-29T00:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
USD_CAD laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
GBP_USD laoded 3999 candles from 2021-10-29T05:00:00.000000000Z to 2022-06-20T23:00:00.000000000Z
GBP_JPY laoded 3999 